### Bitcoin block sampler
[Bitcoin core RPC API](https://en.bitcoin.it/wiki/Original_Bitcoin_client/API_calls_list)

#### Operation ordering


In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is running!')

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin/regtest'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind -chain=regtest PID: {f.read()}')
else:
    raise SystemExit('Bitcoind -chain=regtest is running!')

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
rpc_port_regtest = '18443'
timeout = 300

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Main Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

### Flow
1. Find address set by address apearance type
  - Unlinked addresses
  - Linked addresses

In [ ]:
%%time
### Flow 1
rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', 
                                  timeout=timeout)

sheight = 578670
eheight = 583179

bi = 0
term = 1000
for i in range(sheight, eheight+term, term):
    i = min(eheight+1, i)
    commands = [ ['getblockhash', height] for height in range(bi, i) ]
    block_hashes = rpc_connection.batch_(commands)
    commands = [ ['getblock', h] for h in block_hashes ]
    blocks = rpc_connection.batch_(commands)
    commands = [ ['']]
    ntx = [ block['nTx'] for block in blocks ]

    for block in blocks:
        if get_time(block['time']) < start:
            continue
        elif get_time(block['time']) > end:
            continue
        if sheight > block['height']:
            sheight = block['height']
        if eheight < block['height']:
            eheight = block['height']
    bi = i
    print(f'Reading {i} / {best_block["height"]} ({i/best_block["height"]*100}%)', end='\r')
print('')
print(f'Query-reply done.')
print(f'Start height: {sheight}, End height: {eheight}')